$$\Huge\textbf{Image Classification with CNN}$$

# Imports

In [ ]:
# standard library imports
import os  # Directory and file operations
import time

# installed library imports
import csv  # For saving results
import matplotlib.pyplot as plt  # Plotting library
import numpy as np
from PIL import Image  # For image loading and preprocessing
import torch  # PyTorch main library
import torch.nn as nn  # Neural network modules
import torch.optim as optim  # Optimization algorithms
from torchsummary import summary  # Model summary utility
import torch.utils.data as data  # Data handling utilities
import torchvision.transforms as transforms  # Transformations for image preprocessing
import torchvision.datasets as datasets  # Standard datasets
from torch.utils.data import DataLoader  # Data loading utilities
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc  # Performance metrics

# Global Constants

In [ ]:
# Directories
# Directories
TRAIN_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_dataset\processed_data\without_cross_validation\train'
VAL_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_dataset\processed_data\without_cross_validation\validation'
TEST_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_dataset\processed_data\without_cross_validation\test'
OUTPUT_DIR = r'C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\project_output\cnn'
RESULTS_DIR = os.path.join(OUTPUT_DIR, 'results')
MODELS_DIR = os.path.join(OUTPUT_DIR, 'models')

In [ ]:
# Create directories if they don't exist
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

In [ ]:
# Mean and Standard deviation of the training set 
MEAN = [0.4333, 0.3943, 0.3591]
STD = [0.2445, 0.2401, 0.2347]

# Hyperparameters

In [ ]:
BATCH_SIZE = 32
NUM_EPOCHS = 5
LEARNING_RATE = 0.001
NUM_CLASSES = 5
CLASSES = ['airplane_cabin', 'hockey_arena', 'movie_theater', 'staircase', 'supermarket']

# 1. Data Loading and Preprocessing

In [ ]:
# Compute the mean and standard deviation of the training set (will be used to normalize train, val and test sets)
def compute_mean_std(dataset):
    """
    Compute the mean and standard deviation of a dataset.

    Inputs:
    - dataset (Dataset): A PyTorch dataset.

    Outputs:
    - mean (list): Mean of the dataset.
    - std (list): Standard deviation of the dataset.
    """
    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
    mean = 0.0
    std = 0.0
    for images, _ in loader:
        batch_samples = images.size(0)  # Batch size (the last batch can have smaller size)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    mean /= len(dataset)
    std /= len(dataset)
    return mean, std

In [ ]:
def get_data_loaders(train_dir, val_dir, test_dir, batch_size=BATCH_SIZE):
    """
    Create data loaders for training, validation, and testing.

    Inputs:
    - train_dir (str): Path to the training data directory.
    - val_dir (str): Path to the validation data directory.
    - test_dir (str): Path to the test data directory.
    - batch_size (int): Batch size for the data loaders.


    Outputs:
    - train_loader, val_loader, test_loader (DataLoader): Data loaders for training, validation, and testing.
    """
    # Initial transform to convert images to tensors
    initial_transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    # Load datasets with initial transform
    train_dataset = datasets.ImageFolder(root=train_dir, transform=initial_transform)
    val_dataset = datasets.ImageFolder(root=val_dir, transform=initial_transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=initial_transform)

    # Compute mean and std on the train_dataset
    mean, std = compute_mean_std(train_dataset)

    # Final transform with normalization
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=mean.tolist(), std=std.tolist())
    ])

    # Reload datasets with final transform
    train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

    # Create DataLoaders for each dataset
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, mean, std

# 2. Model Initialization

## 2.1. Weight Initialization

In [ ]:
def weights_init(m):
    """
    Initialize weights with a Gaussian distribution.

    Inputs:
    - m (nn.Module): A neural network module.
    """
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')  # He initialization
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)  # Initialize bias to 0

## 2.2. CNN Architecture

In [ ]:
"""
CNN Architecture:
- Conv Layer 1: 3 input channels, 32 output channels, kernel size 3x3, stride 1, padding 1
- BatchNorm Layer 1: 32 channels
- ReLU Activation 1
- MaxPool Layer 1: kernel size 2x2, stride 2
- Conv Layer 2: 32 input channels, 64 output channels, kernel size 3x3, stride 1, padding 1
- BatchNorm Layer 2: 64 channels
- ReLU Activation 2
- MaxPool Layer 2: kernel size 2x2, stride 2
- Conv Layer 3: 64 input channels, 128 output channels, kernel size 3x3, stride 1, padding 1
- BatchNorm Layer 3: 128 channels
- ReLU Activation 3
- MaxPool Layer 3: kernel size 2x2, stride 2
- Dropout: 0.5
- Fully Connected Layer 1: 128 * 32 * 32 inputs, 512 outputs
- ReLU Activation 4
- Dropout: 0.5
- Fully Connected Layer 2: 512 inputs, 5 class outputs
"""

In [ ]:
class CNN(nn.Module):
    """
    A simple Convolutional Neural Network for image classification.
    """
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # First convolutional layer
            nn.BatchNorm2d(32),  # Batch normalization
            nn.ReLU(inplace=True),  # ReLU activation
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Second convolutional layer
            nn.BatchNorm2d(64),  # Batch normalization
            nn.ReLU(inplace=True),  # ReLU activation
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Third convolutional layer
            nn.BatchNorm2d(128),  # Batch normalization
            nn.ReLU(inplace=True),  # ReLU activation
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling
            nn.Dropout(0.5)  # Dropout for regularization
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 32 * 32, 512),  # Fully connected layer
            nn.ReLU(inplace=True),  # ReLU activation
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(512, num_classes)  # Output layer
        )

    def forward(self, x):
        """
        Forward pass of the network.

        Inputs:
        - x (Tensor): Input image tensor.

        Outputs:
        - x (Tensor): Output logits tensor.
        """
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.classifier(x)
        return x


# 3. Model Training

In [ ]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=NUM_EPOCHS):
    """
    Train the CNN model.

    Inputs:
    - model (nn.Module): The CNN model.
    - criterion (nn.Module): Loss function.
    - optimizer (torch.optim.Optimizer): Optimizer for training.
    - train_loader (DataLoader): DataLoader for training data.
    - val_loader (DataLoader): DataLoader for validation data.
    - num_epochs (int): Number of epochs to train the model.

    Outputs:
    - model (nn.Module): The trained CNN model.
    - history (dict): Dictionary containing training and validation loss and accuracy for each epoch.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 'val_accuracy': []}

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = 100 * correct / total
        history['train_loss'].append(epoch_loss)
        history['train_accuracy'].append(epoch_accuracy)

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = 100 * correct / total
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)

        print(f'Epoch {epoch}/{num_epochs - 1}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}')
    
    return model, history


# 4. Model Evaluation

## 4.1. Performances of the Results

In [ ]:
def evaluate_model(model, data_loader, dataset_type="Test"):
    """
    Evaluate the CNN model on a dataset.

    Inputs:
    - model (nn.Module): The CNN model.
    - data_loader (DataLoader): DataLoader for the data.
    - dataset_type (str): Type of the dataset (Train/Validation/Test).

    Outputs:
    - accuracy (float): Accuracy of the model on the dataset.
    - precision (float): Precision of the model on the dataset.
    - recall (float): Recall of the model on the dataset.
    - f1 (float): F1 score of the model on the dataset.
    - cm (ndarray): Confusion matrix of the model on the dataset.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Compute performance metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    print(f'{dataset_type} Accuracy: {accuracy:.2f}')
    print(f'{dataset_type} Precision: {precision:.2f}')
    print(f'{dataset_type} Recall: {recall:.2f}')
    print(f'{dataset_type} F1-Score: {f1:.2f}')
    print(f'{dataset_type} Confusion Matrix:')
    print(cm)

    return accuracy, precision, recall, f1, cm

## 4.2. Performances Over the Epochs

In [ ]:
def plot_metrics(history, save_path):
    """
    Plot training and validation loss and accuracy over epochs.

    Inputs:
    - history (dict): Dictionary containing training and validation loss and accuracy for each epoch.
    - save_path (str): The file path to save the plot.
    """
    epochs = range(len(history['train_loss']))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_accuracy'], label='Training Accuracy')
    plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.savefig(save_path, dpi=300)  # Increase the resolution with dpi
    plt.show()

In [ ]:
def plot_roc_auc(model, data_loader, num_classes=NUM_CLASSES, save_path=None):
    """
    Plot ROC curve and calculate AUC for each class.

    Inputs:
    - model (nn.Module): The trained CNN model.
    - data_loader (DataLoader): DataLoader for the data.
    - num_classes (int): Number of classes.
    - save_path (str): The file path to save the plot.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    fpr = {}
    tpr = {}
    roc_auc = {}

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(all_labels == i, all_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure()
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='best')
    if save_path:
        plt.savefig(save_path, dpi=300)
    plt.show()


# 5. Save Model and Results

## 5.1. Save the Trained Model

In [ ]:
def save_model(model, epoch, path):
    """
    Save the trained model to a file.

    Inputs:
    - model (nn.Module): The trained CNN model.
    - epoch (int): The epoch at which the model is saved.
    - path (str): The file path to save the model.
    """
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)

## 5.2. Save the Results in a CSV

In [ ]:
def save_metrics_to_csv(metrics, path, hyperparams):
    """
    Save the performance metrics to a CSV file. If the file exists, append to it.

    Inputs:
    - metrics (dict): Dictionary of performance metrics.
    - path (str): The file path to save the metrics.
    - hyperparams (dict): Dictionary of hyperparameters.
    """
    file_exists = os.path.isfile(path)
    with open(path, 'a', newline='') as csvfile:
        fieldnames = ['epoch', 'train_loss', 'val_loss', 'train_accuracy', 'val_accuracy'] + list(hyperparams.keys())
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if not file_exists:
            writer.writeheader()
        
        for epoch in range(len(metrics['train_loss'])):
            row = {
                'epoch': epoch,
                'train_loss': metrics['train_loss'][epoch],
                'val_loss': metrics['val_loss'][epoch],
                'train_accuracy': metrics['train_accuracy'][epoch],
                'val_accuracy': metrics['val_accuracy'][epoch]
            }
            row.update(hyperparams)
            writer.writerow(row)


# 6. Main Execution

In [ ]:
# Create data loaders
train_loader, val_loader, test_loader, mean, std = get_data_loaders(TRAIN_DIR, VAL_DIR, TEST_DIR, BATCH_SIZE)

# Initialize model
model = CNN(num_classes=NUM_CLASSES)

# Initialize the weights
model.apply(weights_init)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
print(mean)
print(std)

In [ ]:
# Train the model and collect training history
trained_model, history = train_model(model, criterion, optimizer, train_loader, val_loader, NUM_EPOCHS)

# Save training metrics to CSV
hyperparams = {
    'batch_size': BATCH_SIZE,
    'num_epochs': NUM_EPOCHS,
    'learning_rate': LEARNING_RATE
}
save_metrics_to_csv(history, os.path.join(RESULTS_DIR, 'training_metrics.csv'), hyperparams)

# Plot and save training and validation metrics
plot_metrics(history, os.path.join(RESULTS_DIR, f'training_metrics_bs{BATCH_SIZE}_lr{LEARNING_RATE}_epochs{NUM_EPOCHS}.png'))

In [ ]:
# Evaluate the model on the training set
print("\nTraining Set Evaluation:")
train_accuracy, train_precision, train_recall, train_f1, train_cm = evaluate_model(trained_model, train_loader, dataset_type="Train")

# Evaluate the model on the validation set
print("\nValidation Set Evaluation:")
val_accuracy, val_precision, val_recall, val_f1, val_cm = evaluate_model(trained_model, val_loader, dataset_type="Validation")

# Evaluate the model on the test set
print("\nTest Set Evaluation:")
test_accuracy, test_precision, test_recall, test_f1, test_cm = evaluate_model(trained_model, test_loader, dataset_type="Test")

# Plot and save ROC and AUC for the test set
plot_roc_auc(trained_model, test_loader, num_classes=NUM_CLASSES, save_path=os.path.join(RESULTS_DIR, f'roc_auc_bs{BATCH_SIZE}_lr{LEARNING_RATE}_epochs{NUM_EPOCHS}.png'))

# Save the trained model
save_model(trained_model, NUM_EPOCHS, os.path.join(MODELS_DIR, f'cnn_bs{BATCH_SIZE}_lr{LEARNING_RATE}_epochs{NUM_EPOCHS}.pth'))

# 7. Project Demo: Image Classification

## 7.1. Load the Best Model

In [ ]:
def load_model(path, model_class, num_classes=NUM_CLASSES):
    """
    Load a trained model from a file.

    Inputs:
    - path (str): The file path to load the model from.
    - model_class (class): The class of the model to instantiate.
    - num_classes (int): Number of output classes.

    Outputs:
    - model (nn.Module): The loaded CNN model.
    - epoch (int): The epoch at which the model was saved.
    """
    start_time = time.time()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model_class(num_classes=num_classes)
    model = model.to(device)
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Model loaded in {time.time() - start_time:.2f} seconds.")
    return model, epoch

## 7.2. Preprocess Chosen Image

In [ ]:
def preprocess_image(image_path, mean, std):
    """
    Preprocess the input image for classification.

    Inputs:
    - image_path (str): Path to the input image.
    - mean (list): Mean for normalization.
    - std (list): Standard deviation for normalization.

    Outputs:
    - image (Tensor): Preprocessed image tensor.
    """
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize the image to 256x256
        transforms.ToTensor(),  # Convert the image to a tensor
        transforms.Normalize(mean=mean, std=std)  # Normalize the image
    ])
    
    image = Image.open(image_path).convert('RGB')  # Open the image and convert to RGB
    image = transform(image)  # Apply the transformations
    image = image.unsqueeze(0)  # Add a batch dimension
    return image

## 7.3. Classify Image

In [ ]:
def classify_image(model, image):
    """
    Classify the input image using the trained model.

    Inputs:
    - model (nn.Module): The trained CNN model.
    - image (Tensor): Preprocessed image tensor.

    Outputs:
    - predicted_class (str): Predicted class label.
    - probability (float): Probability of the predicted class.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    with torch.no_grad():
        image = image.to(device)
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)
        max_prob, predicted_label = torch.max(probabilities, 1)
        predicted_class = CLASSES[predicted_label.item()]
        probability = max_prob.item()

    return predicted_class, probability

## 7.4. Preprocess and Classify an Image

In [ ]:
def preprocess_and_classify_image(model, mean, std, image_path):
    """
    Preprocess an image and classify it using the trained model.

    Inputs:
    - model (nn.Module): The trained CNN model.
    - mean (list): Mean for normalization.
    - std (list): Standard deviation for normalization.
    - image_path (str): Path to the input image.
    """
    print(f"Selected image: {image_path}")
    image = preprocess_image(image_path, mean, std)
    print("Image preprocessing complete.")
    predicted_class, probability = classify_image(model, image)
    print(f"Predicted Class: {predicted_class}, Probability: {probability:.4f}")

In [ ]:
# Load the trained model
model_path = os.path.join(MODELS_DIR, 'cnn_bs32_lr0.001_epochs10.pth')  # Replace with the actual model's filename
loaded_model, saved_epoch = load_model(model_path, CNN)

In [ ]:
# Chosen image path
image_path = r"C:\Users\helen\Documents\Concordia University\summer 2024\COMP 6721\project_code\data\original\places365_standard\val\airplane_cabin\Places365_val_00003919.jpg"

# Classify an image using the loaded model
preprocess_and_classify_image(loaded_model, MEAN, STD, image_path)